In [1]:
import os
import subprocess
from config import Config
print(Config.SDE_CONNECTION)

from arcgis.gis import GIS
from arcgis.features import FeatureLayer

gis = GIS(Config.PORTAL_URL, Config.PORTAL_USER, Config.PORTAL_PASSWORD)


K:/webmaps/basemap/cc-gis.sde


In [ ]:
def read_shape_data(gis):
    key_fields = ['DocumentNa']
    popup_fields = ["SYEAR", "SurveyDate", "BookPage", "Client", "Firm", "SurveyorKe"]

    # Using arcgis, I have to publish on Portal but then I can use spatial data.
    url = "https://delta.co.clatsop.or.us/server/rest/services/Public_Works/Surveys/FeatureServer/0"
    layer = FeatureLayer(url, gis)
    df = layer.query(where="1=1", out_fields=key_fields+popup_fields).sdf

    return df
survey_df = read_shape_data(gis)

In [ ]:
import arcpy
import pandas as pd
images = os.path.join(Config.SDE_CONNECTION, "Clatsop.dbo.SURVEYIMAGES")
n = arcpy.da.TableToNumPyArray(in_table=images, field_names=["DocumentName", "Image", "DateModified"], skip_nulls=False)
docs_df = pd.DataFrame(n)
docs_df


In [ ]:
x = pd.merge(survey_df, docs_df, left_on="DocumentNa", right_on="DocumentName", validate="m:m")
x

In [ ]:
basepath = "\\\\clatsop.co.clatsop.or.us\\data\\applications\\surveyordata\\survey\\scanned surveys\\aa_indexed_surveys\\"
groups = x.groupby('DocumentNa')
surveys2 = {}
for name,group in groups:
    longimages = group["Image"].tolist()
    surveys2[name] = [os.path.splitext(item)[0].lower().replace(basepath,'').replace('\\', '/') for item in longimages]  
surveys2


In [ ]:
surveys3 = list()
baseurl = "https://delta.co.clatsop.or.us/survey/"
maxlink=0
for k,images in surveys2.items():
    link = 0
    d = dict()
    d['DocumentNa'] = k
    for baselink in sorted(images):
        link += 1
        name = os.path.split(baselink)[1]
        d['SURVEY%02d' % link] = name
        d['HYPERLINK%02d' % link] = baseurl + baselink + '.pdf'
    maxlink = max(maxlink, link)
    d['HYPERLINKS'] = maxlink
    surveys3.append(d)
surveys3

In [ ]:
hyperlinks_df=pd.DataFrame.from_dict(surveys3)

In [ ]:
xl = pd.merge(survey_df, hyperlinks_df, on="DocumentNa")
xl

In [ ]:
from utils import get_files_df
import os
os.chdir(Config.SOURCE)
files_df = get_files_df('.')
files_df

In [ ]:
def fixdocname(s):
    pathname = s.Image
    p,fe = os.path.split(pathname)
    f,e = os.path.splitext(fe)
    s['Doc'] = f
    return s
docs_short_df = docs_df.apply(fixdocname, axis=1)
docs_short_df

In [ ]:
xdocs = pd.merge(docs_short_df, files_df, left_on='Doc', right_on='file')
xdocs[xdocs['match']==False]

In [ ]:
import numpy as np
import datetime
dti = pd.to_datetime(
    ["1/1/2018", np.datetime64("2018-01-01"), datetime.datetime(2018, 1, 1)]
)
dti


In [ ]:
from datetime import datetime
floatytime = datetime.fromtimestamp(1566924882.689)
type(floatytime)
pd.to_datetime(floatytime)

In [ ]:
def ddd():
    d = dict()
    d[0] = [sorted([3,2,1]), sorted([])]
    return d
d = ddd()
for item in d:
    print(item, d[item])

In [ ]:
# Print every table entry that does not have a corresponding file on the fileserver.
for index,row in docs_df.iterrows():
    if not os.path.exists(row.Image):
        print(row.Image)

In [ ]:
from requests.utils import requote_uri
requote_uri("https://myurl/file name with spaces.pdf?a=t&t=t")


In [ ]:
from config import Config
import arcpy
arcpy.SignInToPortal(Config.PORTAL_URL, Config.PORTAL_USER, Config.PORTAL_PASSWORD)

project = arcpy.mp.ArcGISProject("K:/webmaps/basemap/basemap.aprx")
maps = project.listMaps()
mapnames = [map.name for map in maps]
map = maps[mapnames.index('Surveys')]
print(map.name)

In [ ]:
layer = map.listLayers()[0]
print(layer.name)

In [ ]:
sd_name = "C:/TEMP/Surveys.sd"
sddraft_name = sd_name + "draft"
server_type = "HOSTING_SERVER"
server_type = "FEDERATED_SERVER"
service_name = 'Surveys'
service_type = "MAP_IMAGE"
#arcpy.mp.CreateWebLayerSDDraft(map, sddraft_name, service_name, 'MY_HOSTED_SERVICES', 'FEATURE_ACCESS', '', True, True)
sddraft = map.getWebLayerSharingDraft(server_type, service_type, service_name, [layer])
sddraft.copyDataToServer = True
sddraft.federatedServerUrl = Config.SERVER_URL
sddraft.exportToSDDraft(sddraft_name)

In [ ]:
arcpy.env.overwriteOutput = True
arcpy.server.StageService(sddraft_name, sd_name)


In [ ]:
from arcgis.gis import GIS
gis = GIS(Config.PORTAL_URL, Config.PORTAL_USER, Config.PORTAL_PASSWORD)
sd = gis.content.add({}, sd_name)
sd

In [ ]:
result = arcpy.server.UploadServiceDefinition(sd_name, Config.SERVER_URL)           
#sd.publish()

In [ ]:
def fn(file):
  ext=['.jpg', '.tif', 'tiff']
  for e in ext:
      if file.lower().endswith(e):
          f = file[0:-len(e)]
          return f + '.pdf'
  return file
fn("foo.JPG.pdf")

In [ ]:
import pandas as pd
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from config import Config
from build_feature_class import read_shape_data
gis = GIS(Config.PORTAL_URL, Config.PORTAL_USER, Config.PORTAL_PASSWORD)


In [ ]:
#url = Config.SURVEY_FEATURES
#key_fields = ['DocumentNa']
#popup_fields = ["SYEAR", "SurveyDate", "BookPage", "Client", "Firm", "SurveyorKe"]
#layer = FeatureLayer(url, gis)
#df = layer.query(where="1=1", out_fields=key_fields+popup_fields).sdf
df = read_shape_data(gis, url=Config.SURVEY_FEATURES)


In [2]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
connUrl = URL.create(
    'mssql+pyodbc',
    username='sde',password='CCgis97103',
    host='cc-gis',
    database='Clatsop',
    query={
        "driver": 'SQL SERVER',
    }
)
engine = create_engine(connUrl)


In [4]:
import pandas as pd
connection = engine.connect()
query = 'SELECT DocumentName,Image FROM [SURVEYIMAGES]'
docs_df=pd.read_sql_query(sql=query, con=connection)
for i in range(10):
    d = docs_df.loc[i]
    print(d)

Image    \\Clatsop.co.clatsop.or.us\data\Applications\S...
Name: 0, dtype: object
Image    \\Clatsop.co.clatsop.or.us\data\Applications\S...
Name: 1, dtype: object
Image    \\Clatsop.co.clatsop.or.us\data\Applications\S...
Name: 2, dtype: object
Image    \\Clatsop.co.clatsop.or.us\data\Applications\S...
Name: 3, dtype: object
Image    \\Clatsop.co.clatsop.or.us\data\Applications\S...
Name: 4, dtype: object
Image    \\Clatsop.co.clatsop.or.us\data\Applications\S...
Name: 5, dtype: object
Image    \\Clatsop.co.clatsop.or.us\data\Applications\S...
Name: 6, dtype: object
Image    \\Clatsop.co.clatsop.or.us\data\Applications\S...
Name: 7, dtype: object
Image    \\Clatsop.co.clatsop.or.us\data\Applications\S...
Name: 8, dtype: object
Image    \\Clatsop.co.clatsop.or.us\data\Applications\S...
Name: 9, dtype: object


In [ ]:
conn = engine.connect()
result = conn.execute("SELECT DocumentName,Image from SURVEYIMAGES")
for row in result:
    print(row)
    break